## PR: Implementation

In [1]:
from datetime import datetime

from IPython.core.debugger import Tracer
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import utils

%matplotlib inline
matplotlib.style.use('ggplot')
tracer = Tracer()

import warnings
warnings.filterwarnings('error')

In [11]:
V = 10
B = 0.9
N = 10000

def gen_data_pr(V, B, N):
    X = np.zeros((N, 3), dtype=int)
    for n in xrange(N):
        p = q = np.random.randint(V)
        while p == q:
            q = np.random.randint(V)
        p, q = sorted([p, q])
        y = np.random.binomial(1, B)
        X[n,:] = np.array([p,q,y])
    return X

X = gen_data_pr(V, B, N)

In [29]:
def get_R_nrm(X):
    V = max(X[:,1]) + 1
    R = utils.get_interactions(X).astype(float)
    R += np.eye(V) * R.sum(axis=0) # Add self-edge equal to sum of victories
    return (R.T / R.T.sum(axis=0)).T

R_nrm = get_R_nrm(X)

In [92]:
def power_method(R_nrm):
    V = R_nrm.shape[0]
    x = np.zeros(V) + 1. / V
    prev = x
    while True:
        prev = x
        x = x.dot(R_nrm)
        if np.linalg.norm((x - prev)) < 0.0001:
            break
    return pd.Series(x).sort_values(ascending=False)

power_method(R_nrm)

9    0.488453
8    0.189298
7    0.105823
6    0.066797
5    0.041658
4    0.033279
3    0.025864
2    0.019651
1    0.016036
0    0.013139
dtype: float64

In [39]:
def np_method(R_nrm):
    eigval, eigvec = np.linalg.eig(R_nrm.T)
    eigval = pd.Series(eigval)
    ev = pd.Series(eigvec[:,eigval.idxmax()])
    return (ev / ev.sum()).sort_values(ascending=False) # Normalize to simplex

np_method(R_nrm)

9    0.488736
8    0.189026
7    0.105814
6    0.066797
5    0.041659
4    0.033278
3    0.025865
2    0.019649
1    0.016038
0    0.013139
dtype: float64

In [87]:
def order_items(X, how='power'):
    R_nrm = get_R_nrm(X)
    if how == 'power':
        return power_method(R_nrm)
    else:
        return np_method(R_nrm)
    
V = 10
B = 0.9
N = 1000
X = gen_data_pr(V, B, N)
out = pd.DataFrame([order_items(X, how='power'), order_items(X, how='np')]).T
out

,0,1
9,(0.495516191074+0j),(0.495697971406+0j)
8,(0.182143873369+0j),(0.181916701279+0j)
7,(0.0895628471125+0j),(0.089580441027+0j)
6,(0.0618146007593+0j),(0.0618206980753+0j)
5,(0.051995572626+0j),(0.0519982329345+0j)
3,(0.028921145955+0j),(0.0289223425322+0j)
1,(0.0277961967184+0j),(0.0278006237555+0j)
4,(0.0274325675971+0j),(0.0274408901277+0j)
0,(0.0191637061386+0j),(0.0191681998678+0j)
2,(0.0156532986505+0j),(0.0156538989947+0j)


In [88]:
def get_error(ordering):
    V = len(ordering)
    return sum(np.abs(out.index - np.arange(V)[::-1]))

get_error(out)

8